# Bledin's compositional truthmaker semantics

## Fineian truthmaker semantics

In [ ]:
{-# LANGUAGE OverloadedLists, FlexibleContexts, DeriveTraversable, ParallelListComp #-}

import Data.Semilattice.Join
import qualified Data.Map as M
import qualified Data.Set as S
import qualified Data.IntSet as IS
import Control.Monad (replicateM)

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedLists, FlexibleContexts, DeriveTraversable, ParallelListComp #-}
Why not:
{-# LANGUAGE OverloadedLists, FlexibleContexts #-}

## State space

Here we just take advantage of the fact that `IntSet` has a `Join` instance; states are sets of integers.

In [15]:
type S = IS.IntSet

Join semilattices give rise to a partial ordering, accessible via the `LessThan` wrapper.

In [14]:
compare (LessThan ([1] :: S)) (LessThan [1,2])

LT

## Syntax

First we recursively define a simple propositional language `Expr`.

In [5]:
data L a = Atom a | Not (L a) | (L a) :&: (L a) | (L a) :|: (L a) deriving (Foldable, Traversable, Functor)

type Expr = L Char

In truthmaker semantics *extensions*, are functions from truth-values to sets of states. This different from the usual presentation, but note that `Bool -> a` is isomorphic to `(a,a)`.

In [5]:
type Ext = Bool -> S.Set S

`variables` just gives back the set of variables in an expression.

In [5]:
variables :: Expr -> S.Set Char
variables = foldr S.insert S.empty

Assignments are functions from variables (i.e., characters) to extensions.

In [16]:
type Assignment = Char -> Ext

Some test data:

In [5]:
test1 = Atom 'a' :&: Atom 'b'

variables test1

fromList "ab"

## Semantics

The basic evaluation algorithm for Fine's sentential truthmaker semantics.

In [22]:
eval :: Assignment -> Expr -> Bool -> S.Set S
eval g (Atom a) = g a
eval g (Not expr) = eval g expr . not
eval g (lconj :&: rconj) = \t ->
        if t
            then
                    let verifiers = (\conj -> S.toList $ eval g conj t)
                    in S.fromList [s \/ s' | s <- verifiers lconj, s' <- verifiers rconj]
            else eval g lconj t `S.union` eval g rconj t
eval g (ldisj :|: rdisj) = \t ->
        if t
            then eval g ldisj t `S.union` eval g rdisj t
            else 
                let falsifiers = (\disj -> S.toList $ eval g disj t)
                in S.fromList [s \/ s' | s <- falsifiers ldisj, s' <- falsifiers rdisj]

In [23]:
testg :: Assignment
testg 'a' True = [[1],[3]]
testg 'a' False = [[2],[4]]
testg 'b' True = [[5],[7]]
testg 'b' False = [[6],[8]]

eval testg test1 True
eval testg test1 False

fromList [fromList [1,5],fromList [1,7],fromList [3,5],fromList [3,7]]

fromList [fromList [2],fromList [4],fromList [6],fromList [8]]